# US Housing Market Analysis: Supply-Demand Dynamics

- Author: [Kiruthikaa](https://github.com/kiruthikaa2512/)
- Repository: [datafun-06-eda](https://github.com/kiruthikaa2512/datafun-04-notebooks)
- Date: June 8th, 2025
- Purpose: The purpose of this notebook is to:
    Apply data analytics skills in a real-world housing market context  
    Analyze features influencing housing supply and demand  
    Create effective visualizations and narratives to communicate insights  
    Demonstrate proficiency with Jupyter notebooks, Python libraries, and EDA workflows  

## Project Overview
This project explores supply and demand trends in the US housing market using a real-world dataset.  
We analyze factors such as price, location, property size, and number of bedrooms/bathrooms to uncover  
market dynamics that may reflect seller or buyer advantage.

We use Python tools such as `pandas`, `seaborn`, and `matplotlib` to clean, explore, and visualize data.  
The goal is to practice data storytelling through well-structured EDA and visual interpretation.



## 1. Import Necessary Libraries  
We will use Python libraries like pandas, Seaborn, and Matplotlib for data analysis and visualization.  


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns   



## 2. Data Loading
We have 2 data CSV sheets which we are going to load below and do a preview and see if they look good to proceed further.

-demand.csv
-supply.csv

In [3]:
# Load the supply dataset
df_supply = pd.read_csv('project_data/supply.csv')
# Load the demand dataset
df_demand = pd.read_csv('project_data/demand.csv')

### 2.1 Initial Data Inspection
Check data types, missing values, and column types.

In [16]:
# Preview the supply dataset
print ("\n---- Preview of the supply dataset -----")
print(df_supply.head())   
# Preview the demand dataset
print ("\n---- Preview of the demand dataset -----")
print(df_demand.head())

# Check the data types of the columns in both datasets
print("\n---- Data types of the supply dataset -----")  
print(df_supply.dtypes)
print("\n---- Data types of the demand dataset -----")  
print(df_demand.dtypes)

# Check the shape of both datasets
print("\n---- Shape of the datasets -----")
print("Shape of supply dataset:", df_supply.shape)
print("Shape of demand dataset:", df_demand.shape)

# Check the info of both datasets
print("\n---- Info of demand dataset -----")
print(df_demand.info())
print("\n---- Info of supply dataset -----")
print(df_supply.info())


# Check for unique values in the supply dataset
print("\n---- Unique values in the supply datasets -----")
print(df_supply.nunique())
# Check for unique values in the demand dataset
print("\n---- Unique values in the demand datasets -----")
print(df_demand.nunique())

# Check for missing values in the supply dataset
print("\n---- Missing values in the supply datasets -----")
print(df_supply.isnull().sum())
# Check for missing values in the demand dataset
print("\n---- Missing values in the demand datasets -----")
print(df_demand.isnull().sum())

# Check for duplicates in the supply dataset
print("\n---- Duplicates in the supply datasets -----")
print(df_supply.duplicated().sum())
# Check for duplicates in the demand dataset
print("\n---- Duplicates in the demand datasets -----")
print(df_demand.duplicated().sum())  


---- Preview of the supply dataset -----
        date  price_index metro_price_index      permits  \
0   1/1/2003      129.321               4.2  1806.333333   
1   1/4/2003      131.756       3.833333333  1837.666667   
2   1/7/2003      135.013       3.633333333  1937.333333   
3  1/10/2003  138.8356667       3.966666667  1972.333333   
4   1/1/2004  143.2986667               3.7  1994.666667   

  construction_spending vacant_units  
0           421328.6667        14908  
1           429308.6667        15244  
2                458890        15614  
3           491437.3333        15654  
4           506856.3333        15895  

---- Preview of the demand dataset -----
        date  price_index  mortgage_rate  sentiment_index  interest_rate  \
0   1/1/2003   129.321000       5.840769        79.966667       2.250000   
1   1/4/2003   131.756000       5.506923        89.266667       2.166667   
2   1/7/2003   135.013000       6.033846        89.300000       2.000000   
3  1/10/2003   13

## Key Observations of Initial Data Exploration
Based off the data inspection output we have couple critical issues to be fixed before we proceed with next sstep of descriptive statisstics. 
1. **Data Types Mismatch in Supply Dataset** - All columns in supply dataset is not treated as numerical values. Example: permits, vacant units etc., are getting treated as strings, mostly could be because of malformed values or could be because the values might have had commas. 
2. **Missing Value in Demand Dataset** - price_index has 1 value missing and interest_rate has 7 values missing. 

### 2.2 Data Cleaning and Preprocessing

As we have observed some missing values and data types mismatch in both Supply and Demand dataset, we are here cleaning up and preprocessisng the data in the below step. 

In [71]:
# Convert relevant columns to numeric first (to avoid issues)
# For df_supply
for col in df_supply.columns:
    df_supply[col] = pd.to_numeric(df_supply[col], errors='coerce')

# For df_demand
for col in df_demand.columns:
    df_demand[col] = pd.to_numeric(df_demand[col], errors='coerce')
# Remove duplicates in both datasets
df_supply = df_supply.drop_duplicates() 
df_demand = df_demand.drop_duplicates()

# Fill missing values with median
df_demand = df_demand.apply(pd.to_numeric, errors='coerce').fillna(df_demand.apply(pd.to_numeric, errors='coerce').median(numeric_only=True))
df_supply = df_supply.apply(pd.to_numeric, errors='coerce').fillna(df_supply.apply(pd.to_numeric, errors='coerce').median(numeric_only=True))

# Round all float columns to 2 decimals in both datasets
for df in [df_demand, df_supply]:
    for col in df.select_dtypes(include='float'):
        df[col] = df[col].round(2)

df_supply['vacant_units'] = df_supply['vacant_units'].fillna(0).astype(int)

# Save cleaned datasets
df_demand.to_csv('project_data/cleaned_demand.csv', index=False)
df_supply.to_csv('project_data/cleaned_supply.csv', index=False)


## Step 3. Initial Descriptive Statistics
Checking basic stats to understand distributions.

In [76]:
# Set display options for better readability
pd.set_option('display.precision', 2)

print("\n---- Summary statistics of the demand dataset -----")
print(df_demand.describe().round(2))

print("\n---- Summary statistics of the supply dataset -----")
# Describe all except vacant_units with rounding
print(df_supply.drop(columns=['vacant_units']).describe().round(2))
# Describe vacant_units as integer summary only once
print(df_supply['vacant_units'].describe().astype(int))





---- Summary statistics of the demand dataset -----
       price_index  mortgage_rate  sentiment_index  interest_rate  \
count        81.00          81.00            81.00          81.00   
mean        180.55           4.70            82.15           1.88   
std          41.16           1.11            12.41           1.68   
min         129.32           2.76            56.10           0.25   
25%         148.21           3.82            73.87           0.75   
50%         172.27           4.44            82.97           1.04   
75%         195.98           5.77            93.07           2.33   
max         303.42           6.66            98.93           6.25   

       median_price       gdp  
count         81.00     81.00  
mean      281104.94  17298.49  
std        68121.52   3851.89  
min       186000.00  11174.13  
25%       228100.00  14448.88  
50%       258400.00  16629.05  
75%       318400.00  19894.75  
max       479500.00  26465.86  

---- Summary statistics of the suppl

## Descriptive Analysis Summary

### Demand Dataset

* Contains 81 records.
* Price index averages 180.55, ranging from 129.3 to 303.4.
* Mortgage rates average 4.7%, ranging from 2.76% to 6.66%.
* Sentiment index averages 82.15, with noticeable variation.
* Interest rates vary widely from 0.25% to 6.25% (mean 1.88%).
* Median house price averages $281K, ranging $186K–$479K.
* GDP averages about 17,300 (units unspecified).

### Supply Dataset

* Contains 82 records.
* Price index similar to demand (180.45).
* Metro price index averages 6.16.
* Building permits average 1,310, ranging 539–2,228.
* Construction spending averages $495K, with high variance.
* Vacant units average 16,900, ranging 0–19,137.

### Notes

* Missing values were filled with medians.
* Non-numeric symbols converted and handled.
* Numeric values rounded to 2 decimals, counts kept as integers.
* Data shows variability across key housing market indicators.

## 4. Initial Data Distribution for Numerical Columns

### Plot Histograms for All Numerical Columns
To understand the spread and skewness of each numerical feature, we use histograms. This helps reveal patterns, outliers, and possible transformations needed.
